## Importing Necessary Libraries

Inculding the glass library for large scale structure simulations.

In [ ]:
import glass.shells
import glass.fields
import warnings
from cosmology import Cosmology
import glass.shells
from scipy.optimize import root_scalar
import camb
from camb import model
import numpy as np
import os
import h5py
from slsim.Halos.halos_plus_glass import (
    read_glass_data,
    skyarea_form_n,
    generate_samples_from_glass,
    halos_plus_glass,
    run_certain_redshift_lensext_kde_by_multiprocessing,
    run_halos_without_kde_by_multiprocessing,
)
from astropy.cosmology import default_cosmology

# glass.ext.camb

This part of the code is modified from the glass.ext.camb. Website: [glass.ext.camb](https://github.com/glass-dev/glass.ext.camb)
This part of GLASS is not easy to be pip install, so we manually copied the code from the website and modified it to fit our needs.

### For this following code-box, credit to the original authors [Dr Nicolas Tessore](https://github.com/ntessore) of the code.


In [3]:
# this is from glass.ext.camb!
def camb_tophat_weight(z):
    """Weight function for tophat window functions and CAMB.

    This weight function linearly ramps up the redshift at low values,
    from :math:`w(z = 0) = 0` to :math:`w(z = 0.1) = 1`.

    """
    return np.clip(z / 0.1, None, 1.0)


def matter_cls(pars, lmax, ws, *, limber=False, limber_lmin=100):
    """Compute angular matter power spectra using CAMB."""

    # make a copy of input parameters so we can set the things we need
    pars = pars.copy()

    # set up parameters for angular power spectra
    pars.WantTransfer = False
    pars.WantCls = True
    pars.Want_CMB = False
    pars.min_l = 1
    pars.set_for_lmax(lmax)

    # set up parameters to only compute the intrinsic matter cls
    pars.SourceTerms.limber_windows = limber
    pars.SourceTerms.limber_phi_lmin = limber_lmin
    pars.SourceTerms.counts_density = True
    pars.SourceTerms.counts_redshift = False
    pars.SourceTerms.counts_lensing = False
    pars.SourceTerms.counts_velocity = False
    pars.SourceTerms.counts_radial = False
    pars.SourceTerms.counts_timedelay = False
    pars.SourceTerms.counts_ISW = False
    pars.SourceTerms.counts_potential = False
    pars.SourceTerms.counts_evolve = False

    sources = []
    for za, wa, _ in ws:
        s = camb.sources.SplinedSourceWindow(z=za, W=wa)
        sources.append(s)
    pars.SourceWindows = sources

    n = len(sources)
    cls = camb.get_results(pars).get_source_cls_dict(lmax=lmax, raw_cl=True)

    for i in range(1, n + 1):
        if np.any(cls[f"W{i}xW{i}"] < 0):
            warnings.warn("negative auto-correlation in shell {i}; improve accuracy?")

    return [cls[f"W{i}xW{j}"] for i in range(1, n + 1) for j in range(i, 0, -1)]

# Set CAMB cosmology and other parameters

the camb cosmology setting part code is from [cambources_test](https://github.com/akrolewski/cambources_test/blob/main/example_for_cosmo_coffee.py), credit to the original authors [ akrolewski (Dr. Adam Krolewski ?)](https://github.com/akrolewski)

we set nside to 2048

In [ ]:
# Code to test limber code for C_{ell} against CAMB's calculations
# CAMB pieces are based off: https://camb.readthedocs.io/en/latest/CAMBdemo.html

# Load fiducial P18 Cosmology
# Results from Planck+BAO in last column of Table 2 in https://arxiv.org/pdf/1807.06209.pdf
ombh2 = 0.02247
tau = 0.0925
ln10As = 3.0589
ns = 0.96824
H0 = 67.7
h = H0 / 100.0
sig8 = 0.8277
# cosmomc_theta =  1.04101/100.

# From sec 3.2 in https://arxiv.org/pdf/1807.06205.pdf
Tcmb = 2.7255
Neff = 3.046
YHe = 0.2454  # None # Set from BBN consistency

# Neutrinos, 1 massive and 2 massless
# omnuh2 = minimal mass / 93.14 eV
# Set minimal mass = 0.06 eV
mnu = 0.00
num_massive_neutrinos = 0
hierarchy = "degenerate"

minkh = 1e-4
maxkh = 1e2
nk = 6000

# we set nisde to 2048
nside = 2048

sky_area = skyarea_form_n(nside=nside, deg2=True)

# Functions to calculate As from sigma8

reverse-calculate As from sigma8 using `scipy.optimize`


In [ ]:
def get_As_from_sigma8(target_sigma8, Om0):
    omch2 = Om0 * 0.677 * 0.677 - 0.02247

    def sigma8_from_ln10As(ln10As):
        _pars = camb.CAMBparams()
        _pars.set_cosmology(
            H0=H0,
            ombh2=ombh2,
            omch2=omch2,
            mnu=mnu,
            neutrino_hierarchy=hierarchy,
            num_massive_neutrinos=num_massive_neutrinos,
            YHe=YHe,
            tau=tau,
        )
        _pars.InitPower.set_params(As=np.exp(ln10As) * 1e-10, ns=ns)
        _pars.set_matter_power(redshifts=[0], kmax=2.0)
        _pars.NonLinear = model.NonLinear_both
        _results = camb.get_results(_pars)
        sigma8 = _results.get_sigma8()[0]
        return sigma8

    # Function to find the discrepancy between the calculated sigma8 and the target sigma8
    def discrepancy(ln10As):
        return sigma8_from_ln10As(ln10As) - target_sigma8

    result = root_scalar(discrepancy, bracket=[2.5, 4], method="brentq")

    # Check if the solution was found
    if result.converged:
        optimal_ln10As = result.root
        print(
            f"The value of ln(10^10 A_s) that results in sigma8 = {target_sigma8} is approximately {optimal_ln10As:.5f}."
        )
    else:
        raise ValueError("Failed to converge to a solution.")

    return np.exp(optimal_ln10As) * 1e-10

# Functions to using GLASS for large scale structure simulations

slghtly modified from the [GLASS documentation examples](https://glass.readthedocs.io/projects/examples/stable/basic/plot_lensing.html)

In [ ]:
def get_glass_cls_file_certain_z_sigma8(z, sigma8, Om0, base_file_path):
    filename = f"cls{z:.1f}_{sigma8:.3f}.npy"
    full_path = os.path.join(base_file_path, filename)

    if os.path.exists(full_path):
        print(f"File {full_path} already exists. Skipping...")
        return

    lmax = 1900
    omch2 = Om0 * 0.677 * 0.677 - 0.02247

    # set up CAMB parameters for matter angular power spectrum
    pars = camb.set_params(
        H0=100 * h,
        omch2=omch2,
        ombh2=ombh2,
        As=get_As_from_sigma8(sigma8, Om0),
        NonLinear=camb.model.NonLinear_both,
    )
    # get the cosmology from CAMB
    cosmo = Cosmology.from_camb(pars)
    # shells of 200 Mpc in comoving distance spacing
    zb = glass.shells.distance_grid(cosmo, 0.0, z, dx=200.0)
    # uniform matter weight function
    # CAMB requires linear ramp for low redshifts
    ws = glass.shells.tophat_windows(zb, weight=camb_tophat_weight)
    # compute angular matter power spectra with CAMB
    cls = matter_cls(pars, lmax, ws)
    np.save(full_path, cls)
    print(os.path.abspath(full_path))


def calculate_glass_data_from_cls_certain_z_sigma8(z, sigma8, Om0, base_file_path):

    filename = f"z{z:.1f}_sigma8_{sigma8:.3f}_data.npy"
    full_path_data = os.path.join(base_file_path, filename)

    if os.path.exists(full_path_data):
        print(f"File {full_path_data} already exists. Skipping...")
        return

    # basic parameters of the simulatio nside = 2048
    lmax = 1900
    omch2 = Om0 * 0.677 * 0.677 - 0.02247
    # set up CAMB parameters for matter angular power spectrum
    pars = camb.set_params(
        H0=100 * h,
        omch2=omch2,
        ombh2=ombh2,
        As=get_As_from_sigma8(sigma8, Om0),
        NonLinear=camb.model.NonLinear_both,
    )

    # get the cosmology from CAMB
    cosmo = Cosmology.from_camb(pars)

    # shells of 200 Mpc in comoving distance spacing
    zb = glass.shells.distance_grid(cosmo, 0.0, z, dx=200.0)

    # uniform matter weight function
    ws = glass.shells.tophat_windows(zb)

    cls_filename = f"cls{z:.1f}_{sigma8:.3f}.npy"
    full_path = os.path.join(base_file_path, cls_filename)

    cls = np.load(full_path)
    # compute Gaussian cls for lognormal fields for 3 correlated shells
    # putting nside here means that the HEALPix pixel window function is applied
    gls = glass.fields.lognormal_gls(cls, nside=nside, lmax=lmax, ncorr=3)

    # generator for lognormal matter fields
    matter = glass.fields.generate_lognormal(gls, nside, ncorr=3)

    convergence = glass.lensing.MultiPlaneConvergence(cosmo)
    kappa_bar = np.zeros(12 * nside**2)
    # main loop to simulate the matter fields iterative
    for i, delta_i in enumerate(matter):

        # add lensing plane from the window function of this shell
        convergence.add_window(delta_i, ws[i])

        # get convergence field
        kappa_i = convergence.kappa

    kappa_bar = kappa_i
    gamma = glass.lensing.from_convergence(kappa=kappa_bar, shear=True)
    gamma_bar = np.abs(gamma)

    data = np.column_stack((kappa_bar, gamma_bar[0]))

    np.save(full_path_data, data)


def calculate_cls_allz_certian_sigma8(sigma8, Om0, base_file_path):
    for z in np.arange(0.1, 5.1, 0.1):
        get_glass_cls_file_certain_z_sigma8(z, sigma8, Om0, base_file_path)


def calculate_data_from_cls_allz_certian_sigma8(sigma8, Om0, base_file_path):
    for z in np.arange(0.1, 5.1, 0.1):
        calculate_glass_data_from_cls_certain_z_sigma8(z, sigma8, Om0, base_file_path)

# Joint Distributions Calculation

Calculates joint distributions of external convergence and shear for combinations of source and deflector redshifts combining both the GLASS and halo rendering approach.
With non-linear correction

In [ ]:
def calculate_joint_distributions(
    zs_range,
    zd_range,
    base_file_path,
    n_iterations=500,
    samples_number=20,
    m_min=1e12,
    m_max=1e16,
    sigma8=0.8277,
    Om0=0.30667,
):
    cosmo = default_cosmology.get()
    joint_distributions = {}

    for zs in zs_range:
        for zd in zd_range:
            if zd < zs:
                file_path = os.path.join(
                    base_file_path, f"z{zs:.1f}_sigma8_{sigma8:.3f}_data.npy"
                )

                kappa_values, gamma_values, _ = read_glass_data(file_name=file_path)
                kappa_random_glass, gamma_random_glass = generate_samples_from_glass(
                    kappa_values, gamma_values, n=1000
                )

                distribution = run_certain_redshift_lensext_kde_by_multiprocessing(
                    n_iterations=n_iterations,
                    sky_area=sky_area,
                    samples_number=samples_number,
                    cosmo=cosmo,
                    m_min=m_min,
                    m_max=m_max,
                    z_max=zs,
                    zs=zs,
                    zd=zd,
                    mass_sheet_correction=False,
                    listmean=False,
                    sigma_8=sigma8,
                    omega_m=Om0,
                )

                nk1 = [point[0] for point in distribution]
                ng1 = [point[1] for point in distribution]
                nk1_mean = sum(nk1) / len(nk1)
                nk1 = [item - nk1_mean for item in nk1]

                total_kappa, total_gamma = halos_plus_glass(
                    kappa_random_glass, gamma_random_glass, nk1, ng1
                )

                kappa_gamma_pairs = np.column_stack((total_kappa, total_gamma))
                joint_distributions[(zs, zd)] = kappa_gamma_pairs

    with h5py.File(
        os.path.join(base_file_path, f"joint_distributions_sigma8_{sigma8:.3f}.h5"), "w"
    ) as f:
        for (zs, zd), kappa_gamma in joint_distributions.items():
            f.create_dataset(f"zs_{zs:.1f}_zd_{zd:.1f}", data=kappa_gamma)

    return joint_distributions

# Calculate distributions without line of sight

Calculates distributions of external convergence and shear for source redshifts combining both the GLASS and halo rendering approach.
Without non-linear correction

In [ ]:
def calculate_no_los_distributions(
    zs_range,
    base_file_path,
    n_iterations=500,
    samples_number=20,
    m_min=8e11,
    m_max=1e16,
    sigma8=0.8277,
    Om0=0.30667,
):
    cosmo = default_cosmology.get()
    joint_distributions = {}

    filename = f"no_nonlinear_distributions_sigma8_{sigma8:.3f}.h5"
    full_path_data = os.path.join(base_file_path, filename)

    if os.path.exists(full_path_data):
        print(f"File {full_path_data} already exists. Skipping...")
        return

    for zs in zs_range:
        file_path = os.path.join(
            base_file_path, f"z{zs:.1f}_sigma8_{sigma8:.3f}_data.npy"
        )

        kappa_values, gamma_values, _ = read_glass_data(file_name=file_path)
        kappa_random_glass, gamma_random_glass = generate_samples_from_glass(
            kappa_values, gamma_values, n=1000
        )

        distribution = run_halos_without_kde_by_multiprocessing(
            n_iterations=n_iterations,
            sky_area=sky_area,
            samples_number=samples_number,
            cosmo=cosmo,
            m_min=m_min,
            m_max=m_max,
            z_max=zs,
            mass_sheet_correction=False,
            listmean=False,
            sigma_8=sigma8,
            omega_m=Om0,
        )

        nk1 = distribution[0]
        ng1 = distribution[1]
        nk1_mean = sum(nk1) / len(nk1)
        nk1 = [item - nk1_mean for item in nk1]

        total_kappa, total_gamma = halos_plus_glass(
            kappa_random_glass, gamma_random_glass, nk1, ng1
        )

        kappa_gamma_pairs = np.column_stack((total_kappa, total_gamma))
        joint_distributions[zs] = kappa_gamma_pairs

    with h5py.File(
        os.path.join(
            base_file_path, f"no_nonlinear_distributions_sigma8_{sigma8:.3f}.h5"
        ),
        "w",
    ) as f:
        for zs, kappa_gamma in joint_distributions.items():
            f.create_dataset(f"zs_{zs:.1f}", data=kappa_gamma)

    return joint_distributions

In [ ]:
sigma8 = 0.825
Om0 = 0.280

# sigma8 = 0.8102
# Om0 = 0.3111

# Get the results out

In [ ]:
base_file_path = "/home/xitang/slsim/data/glass"
calculate_cls_allz_certian_sigma8(sigma8=sigma8, Om0=Om0, base_file_path=base_file_path)
calculate_data_from_cls_allz_certian_sigma8(
    sigma8=sigma8, Om0=Om0, base_file_path=base_file_path
)

In [ ]:
base_file_path = "/home/xitang/slsim/data/glass"
zs_range = np.arange(0.1, 5.0, 0.1)
zd_range = np.arange(0.1, 5.0, 0.1)
joint_distributions = calculate_joint_distributions(
    zs_range=zs_range,
    zd_range=zd_range,
    base_file_path=base_file_path,
    sigma8=sigma8,
    Om0=Om0,
)

In [ ]:
base_file_path = "/home/xitang/slsim/data/glass"
zs_range = np.arange(0.1, 5.0, 0.1)
joint_distributions2 = calculate_no_los_distributions(
    zs_range=zs_range, base_file_path=base_file_path, sigma8=sigma8, Om0=Om0
)